In [1]:
from protera_stability import EmbeddingExtractor1D

In [2]:
from protera_stability import open_train_test 
from pathlib import Path

data_path = Path("../data")

> * Los `.csv` deben contener las columnas `labels` y `sequences`.

> * `data_path` debe ser Path y un directorio que contenga tus `.csv`

In [3]:
sets = open_train_test(data_path, "stability")
sets["train"]

,labels,sequence
0,0.37,GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT
1,0.62,TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK
2,-0.03,TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK
3,1.41,GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT
4,1.11,TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT
...,...,...
7705,0.80,GSSKTQYEYDTKEEHQKAYEKFKKQGIPVTITQKNGKWFVQVE
7706,0.82,TIDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKPY
7707,0.66,TQDEIIKALEQAVKDNKPIQVGNYTVTSADEAEKLAKKLKKEY
7708,1.05,TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVWVTR


In [4]:
args_dict = {
    "model_name": "esm1b_t33_650M_UR50S",
    "base_path": data_path,
    "gpu": True,
}

emb_stabilty = EmbeddingExtractor1D(**args_dict)

Using cache found in /home/roberto/.cache/torch/hub/facebookresearch_esm_master


In [5]:
embeddings = emb_stabilty.generate_embeddings(
    ["stability.csv"],  # esta en data_path
    path_out="stability_embeddings",  # guardarlo en data_path/stability_train.pkl
    bs=256,
    overwrite=True,
    target_name="stability"
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [01:07<00:00,  1.65s/it]

Embeddings saved to ../data/stability_embeddings.pkl.pkl


In [5]:
emb_stabilty.generate_datasets?

In [8]:
dset = emb_stabilty.generate_datasets(
    ["stability.csv"],
    h5_stem="stability",  # data_path / "stability".h5
    bs=256,
    embedding_file="stability_embeddings",  # cargar los embeddings desde donde se espera que esten
    target_name="stability_scores"
)

AttributeError: 'EmbeddingExtractor1D' object has no attribute 'data_path'

In [12]:
import pickle

In [13]:
pickle.load(open("../data/stability_embeddings.pkl", "rb"))

{'GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT': array([ 0.0095719 ,  0.0721352 ,  0.03456477, ..., -0.06875882,
        -0.01336848,  0.08879898], dtype=float32),
 'TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK': array([ 0.1338777 ,  0.1214103 ,  0.05970053, ..., -0.11711021,
        -0.0255498 ,  0.0046577 ], dtype=float32),
 'TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK': array([ 0.01909522,  0.1302496 ,  0.03853326, ..., -0.15073481,
         0.00366706,  0.1114075 ], dtype=float32),
 'GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT': array([-0.01762303,  0.0710905 ,  0.06716064, ..., -0.06012489,
         0.01815582,  0.11649472], dtype=float32),
 'TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT': array([-0.07225917,  0.16300306, -0.00242234, ..., -0.07418614,
         0.08749937,  0.23511803], dtype=float32),
 'SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG': array([ 0.00252395, -0.04101524,  0.05300456, ..., -0.03823757,
         0.03643397,  0.09464986], dtype=float32),
 'TIDEIIKALEQAVKDGKPIQ

In [15]:
dset["labels"][:10], dset["sequences"][:10]

(array([ 0.37,  0.62, -0.03,  1.41,  1.11,  1.24,  1.05,  0.89,  0.88,
         1.15]),
 array([b'GSSQETIEVEDEEEARRVAKELRKKGYEVKDERRGNKWHVHRT',
        b'TLDEARELVERAKKEGTGMDVNGQRFEDWREAERWVREQEKNK',
        b'TELKKKLEEALKKGEEVRVKFNGIEIRNTSEDAARKAVELLEK',
        b'GSSQETIEVEDEEEARRVAKELRKTGYEVKIERRGNKWHVHRT',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLAKKYNLQVTIKNGKITVT',
        b'SKDEAQREAERAIRSGNKEEARRILEEVGYSPEQAERIIRKLG',
        b'TIDEIIKALEQAVKDGKPIQVGNYTVTSADEAEKLAKKLKKEY',
        b'FEIPDDVPLPAGWEMARTSSGQRYFKNHIDQTTTWQDPRKAMLSQM',
        b'TTIHVGDLTLKYDNPKKAYEIAKKLDKKYNLTVTIKNGKITVT',
        b'GSSGSLSDEDFKAVFGMTRSAFAMLPLWKQQNLKKEKGLFGSS'], dtype=object))

In [16]:
dset.close() # hay que cerrarlo pq es un h5py